# CIBERSORTx Tutorial
```
Andrew E. Davidson
aedavids@ucsc.edu
8/23/2022
```

Goal play round with ciber sort

In [62]:
import numpy as np
import pandas as pd

## Create Signature Gene file

In [63]:
m = 20 # number of samples
n =  8 # number of genes
k =  3 # number of types

In [67]:
geneProfileList = []
for i in range(k):
    tmp = np.zeros(n)
    for j in range(i, 1 + k):
        tmp[i + j] = 1
    geneProfileList.append(tmp)
    
geneProfileList[1][n -1] = 1
geneProfileList

[array([1., 1., 1., 1., 0., 0., 0., 0.]),
 array([0., 0., 1., 1., 1., 0., 0., 1.]),
 array([0., 0., 0., 0., 1., 1., 0., 0.])]

In [71]:
geneNames = [ "G" + str(i) for i in range(n)]
typeNames = [ "T" + str(i) for i in range(k)]

geneProfileDict = { 'name': geneNames}
signGeneDF = pd.DataFrame( geneProfileDict )

for i in range(len(typeNames)):
    typeName = typeNames[i]
    signGeneDF[typeName] = geneProfileList[i]

signGeneDF

,name,T0,T1,T2
0,G0,1.0,0.0,0.0
1,G1,1.0,0.0,0.0
2,G2,1.0,1.0,0.0
3,G3,1.0,1.0,0.0
4,G4,0.0,1.0,1.0
5,G5,0.0,0.0,1.0
6,G6,0.0,0.0,0.0
7,G7,0.0,1.0,0.0
